# First, let us set up a few dependencies

Don't forget to switch to a GPU-enabled colab runtime!

```
Runtime -> Change Runtime Type -> GPU
```

In [22]:
import os
import contextlib
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

# This mounts your google drive to this notebook. You might have to change the path to fit with your dataset folder inside your drive.

Read the instruction output by the cell bellow carefully!

In [ ]:
# Create a temporary workspace
import tempfile


SESSION_WORKSPACE = tempfile.mkdtemp()
print(f"Session workspace created at: {SESSION_WORKSPACE}")

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/My Drive"

In [ ]:
# Unzip the dataset
import shutil
import os

DATASET_DIR_NAME = "killmeWithDuckies"
DATASET_ZIP_NAME = f"{DATASET_DIR_NAME}.zip"
DATASET_DIR_PATH = os.path.join(SESSION_WORKSPACE, DATASET_DIR_NAME)
TRAIN_DIR = "train"
VALIDATION_DIR = "valid"
IMAGES_DIR = "images"
LABELS_DIR = "labels"


def show_info(base_path: str):
  for l1 in [TRAIN_DIR, VALIDATION_DIR]:
    for l2 in [IMAGES_DIR, LABELS_DIR]:
      p = os.path.join(base_path, l1, l2)
      print(f"#Files in {l1}/{l2}: {len(os.listdir(p))}")


def unzip_dataset():
  # check zipped file
  zip_path = os.path.join(DRIVE_PATH, DATASET_ZIP_NAME)
  assert os.path.exists(zip_path), f"No zipped dataset found at {zip_path}! Abort!"

  # unzip the data
  print("Unpacking zipped data...")
  shutil.unpack_archive(zip_path, DATASET_DIR_PATH)
  print(f"Zipped dataset unpacked to {DATASET_DIR_PATH}")

  # show some info
  show_info(DATASET_DIR_PATH)


unzip_dataset()

In [ ]:
# change  working directory to the session workspace
os.chdir(SESSION_WORKSPACE)
print(f"PWD: {os.getcwd()}")

# install pytorch and torchvision (these versions must match those on the physical duckiebot)
!pip3 install torch==1.11 torchvision==0.12.0

# Next, we will clone Yolov5

In [ ]:
!rm -rf ./yolov5
!git clone https://github.com/ultralytics/yolov5.git -b v7.0
!cd yolov5 && pip3 install -r requirements.txt

# We now inform the training process of the format and location of our dataset

In [ ]:
%%writefile ./yolov5/data/data.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../killmeWithDuckies/train
val: ../killmeWithDuckies/valid

# number of classes
nc: 7

# class names
names:
  0: Speed Limit 20 KMPh
  1: Speed Limit 30 KMPh
  2: Speed Limit 50 KMPh
  3: Stop_Sign
  4: Turn right ahead
  5: Turn left ahead
  6: Duckie

# And we're ready to train! This step will take about 5 minutes.

Notice that we're only training for 10 epochs. That's probably not enough!

In [ ]:

!cd yolov5 && python3 train.py --cfg ./models/yolov5n.yaml --img 416 --batch 50 --epochs 100 --data data.yaml --weights yolov5n.pt --cache ram --cache disk

In [ ]:
import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")
print(f"Marked the model from the latest run ({latest_exp}) as yolov5/best.pt.")

In [ ]:
#!cd yolov5 && python3 detect.py --weights best.pt --source ../duckietown_object_detection_dataset/val/images/
!cd yolov5 && python3 detect.py --weights best.pt --img 416 --source ../killmeWithDuckies/valid/images/